In [ ]:
!pip install timm

In [1]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import timm # PyTorch Image Models

from glob import glob

import torchvision.datasets   as datasets
import torchvision.transforms as transforms

import torch.optim as optim

from torch.utils.data        import DataLoader, Dataset
from sklearn.model_selection import StratifiedKFold

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') #GPU 할당
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
label_df = pd.read_csv('./data/train.csv')
label_df.head()

label_df['label'][label_df['label'] == '10-1'] = 10 ## label : 10-1 -> 10
label_df['label'][label_df['label'] == '10-2'] = 0 ## Label : 10-2 -> 0
label_df['label'] = label_df['label'].apply(lambda x : int(x)) ## Dtype : object -> int

In [4]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))

    # get label
    label_list.extend(label_df['label'])
                
    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []
    
    # get image path
    img_path_list.extend(glob(os.path.join(data_dir, '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))

    return img_path_list

In [5]:
all_img_path, all_label = get_train_data('./train')
test_img_path = get_test_data('./test')

In [6]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index): #index번째 data를 return
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self): #길이 return
        return len(self.img_path_list)

In [7]:
IMAGE_SIZE = 528
BATCH_SIZE = 16
CLASS_N = 11
LEARNING_RATE = 1e-3
EPOCHS = 200
NUM_WORKERS = 16

In [8]:
#Set data preprocessing

train_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize([IMAGE_SIZE, IMAGE_SIZE]),
                    transforms.ToTensor(),
                    transforms.Normalize([0.4857, 0.5401, 0.5882], [0.2033, 0.1861, 0.1771]),
                    transforms.RandomRotation(20, interpolation=cv2.INTER_CUBIC),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomPerspective(distortion_scale=.15,p=.15,interpolation=transforms.InterpolationMode.NEAREST),
                    ])

test_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize([IMAGE_SIZE, IMAGE_SIZE]),
                    transforms.ToTensor(),
                    transforms.Normalize([0.4857, 0.5401, 0.5882], [0.2033, 0.1861, 0.1771]),
                    ])

C:\Users\sangt\anaconda3\lib\site-packages\torchvision\transforms\transforms.py:1300: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [12]:
train_dataset = CustomDataset(all_img_path, all_label, train_mode=True, transforms=train_transform)
meanstd_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS)

print('==> Computing mean and std..')
mean = torch.zeros(3)
std = torch.zeros(3)

for inputs, _labels in meanstd_loader:
    for i in range(3):
        mean[i] += inputs[:,i,:,:].mean()
        std[i] += inputs[:,i,:,:].std()
mean.div_(len(meanstd_loader))
std.div_(len(meanstd_loader))
print(mean, std)

==> Computing mean and std..
tensor([nan, nan, nan]) tensor([nan, nan, nan])


In [14]:
train_dataset = CustomDataset(all_img_path, all_label, train_mode=True, transforms=train_transform) 
for i in range(3): 
    train_dataset += CustomDataset(all_img_path, all_label, train_mode=True, transforms=train_transform)
    all_img_path = all_img_path + all_img_path + all_img_path + all_img_path
    all_label = all_label + all_label + all_label + all_label



In [15]:
print(all_img_path)
print(all_label)

[]
[0, 10, 3, 8, 9, 9, 8, 0, 5, 5, 9, 2, 4, 10, 9, 2, 0, 7, 4, 2, 6, 10, 10, 6, 10, 5, 6, 3, 0, 8, 1, 2, 8, 2, 5, 10, 6, 1, 1, 4, 4, 0, 7, 4, 4, 9, 0, 9, 8, 2, 6, 10, 9, 5, 3, 3, 6, 2, 1, 2, 10, 0, 5, 3, 4, 6, 6, 7, 2, 8, 6, 0, 9, 0, 6, 3, 8, 0, 1, 7, 6, 1, 6, 9, 6, 7, 3, 10, 3, 10, 10, 4, 3, 1, 6, 6, 2, 6, 9, 7, 9, 4, 2, 7, 10, 4, 3, 2, 6, 4, 3, 10, 9, 2, 7, 0, 8, 10, 5, 5, 6, 0, 9, 0, 7, 6, 2, 0, 1, 5, 5, 5, 4, 6, 4, 9, 10, 6, 1, 7, 2, 9, 4, 3, 1, 4, 3, 6, 2, 10, 7, 10, 1, 4, 1, 6, 4, 5, 5, 2, 5, 6, 9, 8, 7, 4, 3, 7, 1, 9, 10, 2, 6, 7, 3, 7, 6, 9, 2, 2, 10, 8, 8, 9, 8, 4, 4, 5, 4, 10, 10, 6, 1, 6, 4, 1, 4, 6, 1, 4, 6, 2, 3, 8, 1, 5, 9, 10, 9, 3, 7, 1, 4, 8, 2, 10, 6, 5, 7, 10, 4, 1, 5, 7, 4, 2, 7, 1, 9, 1, 8, 4, 6, 0, 4, 6, 2, 3, 7, 2, 2, 1, 0, 1, 2, 3, 8, 2, 1, 10, 1, 5, 2, 5, 7, 6, 8, 4, 1, 1, 3, 5, 8, 8, 0, 7, 9, 10, 9, 3, 4, 2, 6, 0, 9, 5, 5, 7, 5, 7, 2, 8, 6, 0, 1, 0, 2, 9, 7, 9, 0, 2, 5, 0, 10, 4, 10, 9, 5, 4, 2, 8, 4, 2, 8, 2, 8, 3, 2, 6, 8, 10, 10, 3, 8, 7, 3, 2, 5, 6, 2, 8, 

In [ ]:
def train(model, optimizer, train_loader, scheduler, device, k_idx, vali_loader, val_num): 
    model.to(device)
    n = len(train_loader)
    
    criterion = torch.nn.CrossEntropyLoss().to(device)
    best_acc  = 0
    best_loss = 11111
    
    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
            
        for img, label in iter(train_loader):
            img, label = img.to(device), label.to(device)
            optimizer.zero_grad()
        
            logit = model(img)
            loss = criterion(logit, label)
            
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))        
        if scheduler is not None: scheduler.step()

        #Validate
        model.eval()
        vali_loss = 0.0
        correct = 0
        with torch.no_grad():
            for img, label in iter(vali_loader):
                img, label = img.to(device), label.to(device)

                logit = model(img)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)
                correct += pred.eq(label.view_as(pred)).sum().item()
        vali_acc = 100 * correct / val_num
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, val_num, 100 * correct / val_num))

        #베스트 모델 저장
        if best_loss > vali_loss:
            best_loss = vali_loss
            torch.save(model.state_dict(), './best_model_{}.pth'.format(k_idx))
            print('Model Saved.')

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True)
for idx, [train_idx, valid_idx] in enumerate(kf.split(X=all_img_path, y=all_label)):
    model = timm.create_model('tf_efficientnet_b6_ns', pretrained=False, num_classes=CLASS_N).to(device)
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1)

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS, sampler=train_subsampler)
    valid_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, num_workers=NUM_WORKERS, sampler=valid_subsampler)
    #train(model, optimizer, train_loader, scheduler, device, idx, valid_loader, len(valid_subsampler))

In [ ]:
def predict(model, test_loader, device, transforms):
    model.eval()
    model_pred = []
    
    with torch.no_grad():
        for img in iter(test_loader):
            img = img.to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

checkpoint = torch.load('./best_model_0.pth')
model.load_state_dict(checkpoint)
preds = predict(model, test_loader, device, transforms)

## TTA, ENSEMBLE, ...
#tta = 3
#preds = []
#for tta_idx in range(tta): preds.append(predict(model, test_loader, device, transforms))
#preds = np.array(preds)
#preds = np.mean(preds, axis=0)

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['label'] = preds#.astype(int)

submission['label'][submission['label'] == 10] = '10-1' ## label : 10 -> '10-1'
submission['label'][submission['label'] == 0] = '10-2' ## Label : 0 -> '10-2'
submission['label'] = submission['label'].apply(lambda x : str(x)) ## Dtype : int -> object

submission.to_csv('submit.csv', index=False)